In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from random import random, seed

%matplotlib inline

In [23]:
# 2D design matrix
# omit y for a 1D design matrix
def DesignMatrix(p_order, x, y = None):
    n = np.sum(range(p_order+2))
    
    if y != None:
        X = np.ones((len(x), 1))
        for k in range(1, p_order+1):
            j = k
            i = 0
            while i < k + 1:
                X = np.column_stack((X, x**i*y**j))
                # if i == j:
                #     X = np.column_stack((X, x**i*y**j))
                # else:
                #     X = np.column_stack((X, x**i*y**j))
                #     X = np.column_stack((X, y**i*x**j))
                j -= 1
                i += 1
    
    else:
        X = np.ones((len(x), p_order+1))
        for i in range(1, p_order+1):
            X[:,i] = x**i

    return X
    # 1                     #1
    # x,y                   #2
    # x²,y²,xy              #3
    # x³,y³,yx²,xy²         #4
    # x⁴,y⁴,yx³,xy³,x²y²    #5
    
    # 1,
    # 2, 2
    # 4, 4, 4
    # 8, 8, 8, 8
    # 16, 16, 16, 16, 16

In [24]:
x = np.array([2])
y = np.array([2])
DesignMatrix(6, x, y)

array([[ 1.,  2.,  2.,  4.,  4.,  4.,  8.,  8.,  8.,  8., 16., 16., 16.,
        16., 16., 32., 32., 32., 32., 32., 32., 64., 64., 64., 64., 64.,
        64., 64.]])

In [25]:
fig = plt.figure()
ax = fig.gca(projection='3d')

# Make data.
x = np.arange(0, 1, 0.05)
y = np.arange(0, 1, 0.05)
x, y = np.meshgrid(x,y)


def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4


z = FrankeFunction(x, y)

# Plot the surface.
surf = ax.plot_surface(x, y, z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Customize the z axis.
ax.set_zlim(-0.10, 1.40)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

TypeError: FigureBase.gca() got an unexpected keyword argument 'projection'

<Figure size 640x480 with 0 Axes>